In [19]:
import platformdirs
from pathlib import Path
import torch
import numpy as np

import onnxruntime

from katago_onnx.utils import load_model
from katago_onnx.utils import load_sgf
from katago_onnx.utils import featurize

cache_dir = Path(platformdirs.user_cache_dir("katago-onnx"))

# Load model
network_name = "kata1-b28c512nbt-adam-s11165M-d5387M"
model_path = cache_dir / network_name / "model.ckpt"
model = load_model(model_path, device="cpu")

onnx_model_path = cache_dir / network_name / "model.onnx"

In [20]:
# Prepare inputs for ONNX export
bin_input = torch.randn(1, 22, 19, 19, dtype=torch.float32)
global_input = torch.randn(1, 19, dtype=torch.float32)
model_inputs = (bin_input, global_input)

# Define input and output names
input_names = ["bin_input", "global_input"]
output_names = [
    "policy",
    "value",
    "miscvalue",
    "moremiscvalue",
    "ownership",
    "scoring",
    "futurepos",
    "seki",
    "scorebelief",
]

# Define dynamic axes (for dynamo=False)
dynamic_axes = {
    "bin_input": {0: "batch_size", 2: "height", 3: "width"},
    "global_input": {0: "batch_size"},
    "policy": {0: "batch_size", 2: "moves"},
    "value": {0: "batch_size"},
    "miscvalue": {0: "batch_size"},
    "moremiscvalue": {0: "batch_size"},
    "ownership": {0: "batch_size", 2: "height", 3: "width"},
    "scoring": {0: "batch_size", 2: "height", 3: "width"},
    "futurepos": {0: "batch_size", 2: "height", 3: "width"},
    "seki": {0: "batch_size", 2: "height", 3: "width"},
    "scorebelief": {0: "batch_size"},
}

# Export the model to ONNX
# Note: We use dynamo=False because dynamo=True currently fails with
# "No ONNX function found for aten.sym_size.int" for this model's dynamic shapes.
torch.onnx.export(
    model,
    model_inputs,
    onnx_model_path,
    input_names=input_names,
    output_names=output_names,
    dynamic_axes=dynamic_axes,
    opset_version=17,
    dynamo=False,
)

/var/folders/fk/ty303kmn61x3y013qnk89g3h0000gn/T/ipykernel_53205/4271510401.py:38: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(
/Users/hadim/Code/libs/katago-onnx/src/katago/train/model_pytorch.py:309: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not genera

In [21]:
target_move = 8

# Path to SGF
sgf_path = "../sgf/game-19-white-38.sgf"
sgf_path = "../sgf/game-13-white-12.sgf"
# sgf_path = "../sgf/game-9-white-19.sgf"

# Load game state from SGF
gamestate = load_sgf(sgf_path, target_move=target_move)

# Featurize the game state
features, bin_input, global_input = featurize(gamestate, model)

# Prepare  inputs for ONNX export
model_inputs = (bin_input, global_input)
onnx_inputs = [tensor.numpy(force=True) for tensor in model_inputs]

ort_session = onnxruntime.InferenceSession(
    onnx_model_path,
    providers=["CPUExecutionProvider"],
)

onnxruntime_input = {
    input_arg.name: input_value for input_arg, input_value in zip(ort_session.get_inputs(), onnx_inputs)
}

# ONNX Runtime returns a list of outputs
onnxruntime_outputs = ort_session.run(None, onnxruntime_input)

# Unpack outputs
policy_logits = np.array(onnxruntime_outputs[0])
value_logits = np.array(onnxruntime_outputs[1])

# Process Policy
# policy_logits is already numpy from ONNX Runtime
# Shape: [batch, num_policy_outputs, num_moves]
# We want the first policy output (index 0) and softmax over moves (axis 2)
policy_probs = np.exp(policy_logits[:, 0, :]) / np.sum(np.exp(policy_logits[:, 0, :]), axis=1, keepdims=True)
policy_probs = policy_probs[0]

# Process Value (Winrate)
# value_logits is already numpy from ONNX Runtime
# Shape: [batch, 3] corresponding to [win, loss, no_result]
value_probs = np.exp(value_logits) / np.sum(np.exp(value_logits), axis=1, keepdims=True)
value_probs = value_probs[0]

winrate = value_probs[0]  # Assuming index 0 is win for current player? Or black? Or white?

# Process Score Lead
# miscvalue_logits is at index 2
miscvalue_logits = np.array(onnxruntime_outputs[2])
# Index 2 of miscvalue is lead
score_lead = miscvalue_logits[0, 2] * model.lead_multiplier

print(f"Winrate: {winrate}")
print(f"Score Lead: {score_lead}")

for a in onnxruntime_outputs:
    print(a.shape)

Loaded SGF with 117 moves.
Game size: 13
Replayed to move 8
Winrate: 0.5851287245750427
Score Lead: 0.2296016365289688
(1, 6, 170)
(1, 3)
(1, 10)
(1, 8)
(1, 1, 13, 13)
(1, 1, 13, 13)
(1, 2, 13, 13)
(1, 4, 13, 13)
(1, 842)
(1, 6, 170)
(1, 3)
(1, 10)
(1, 8)
(1, 1, 13, 13)
(1, 1, 13, 13)
(1, 2, 13, 13)
(1, 4, 13, 13)
(1, 842)
Winrate: 0.5851287245750427
Score Lead: 0.2296016365289688
(1, 6, 170)
(1, 3)
(1, 10)
(1, 8)
(1, 1, 13, 13)
(1, 1, 13, 13)
(1, 2, 13, 13)
(1, 4, 13, 13)
(1, 842)
(1, 6, 170)
(1, 3)
(1, 10)
(1, 8)
(1, 1, 13, 13)
(1, 1, 13, 13)
(1, 2, 13, 13)
(1, 4, 13, 13)
(1, 842)


In [18]:
winrate

np.float32(0.5851287)